In [1]:
#This block is for initializing variables
import pickle
from pprint import pprint as pp
import datetime
import glob
import random
import os
from textblob import TextBlob
import pandas as pd

N_TURN = 3
list1=[]
bot_strategies=[]
user_utterances=[]
#state action table
state_action_dict=pd.DataFrame.from_items([('qs+b', [0.0, 0.0, 0.0,0.0]), ("qs+e", [0.0, 0.0, 0.0,0.0]),("qs-b", [0.0, 0.0, 0.0,0.0]),("qs-e", [0.0, 0.0, 0.0,0.0]),
                                           ("qs'b", [0.0, 0.0, 0.0,0.0]),("qs'e", [0.0, 0.0, 0.0,0.0]),("q'ls+b", [0.0, 0.0, 0.0,0.0]),("q'ls+e", [0.0, 0.0, 0.0,0.0]),
                                            ('qls-b', [0.0, 0.0, 0.0,0.0]), ("q'ls-e", [0.0, 0.0, 0.0,0.0]),("q'ls'b", [0.0, 0.0, 0.0,0.0]),("q'ls'e", [0.0, 0.0, 0.0,0.0]),
                                           ("q'l's+b", [0.0, 0.0, 0.0,0.0]),("q'l's+e", [0.0, 0.0, 0.0,0.0]),("q'l's-b", [0.0, 0.0, 0.0,0.0]),("q'l's-e", [0.0, 0.0, 0.0,0.0]),
                                          ("q'l's'b", [0.0, 0.0, 0.0,0.0]),("q'l's'e", [0.0, 0.0, 0.0,0.0])],
                                            orient='index', 
                                          columns=['affirmative', 'opinion', 'negative','question'])

#temporary dictionary which stores the count of the recorded utterance till date,this is used for calculating the average to be stored in state table
temp_dict={"qs+b":0,"qs+e":0,"qs-b":0,"qs-e":0,"qs'b":0,"qs'e":0,"q'ls+b":0,"q'ls+e":0,"q'ls-b":0,"q'ls-e":0,"q'ls'b":0,"q'ls'e":0,
           "q'l's+b":0,"q'l's+e":0,"q'l's-b":0,"q'l's-e":0,"q'l's'b":0,"q'l's'e":0}

In [4]:
#this is the training data on which we initially learn
list1=[]
user_utterances=[]
bot_strategies=[]
#function which updates the state value table for each conversation where list1 is the utterance of the user and list2 is the strategy
#applied by the bot
def record_check(list1,list2,evaluated_score):
    i=0
    for key in list1:
        #print state_action_dict.at[key,list2[i]]
        previous_value=state_action_dict.at[key,list2[i]]
        temp_dict[key]=temp_dict[key]+1
        print(float(evaluated_score))
        #print temp_dict[key],((previous_value*(temp_dict[key]-1))+evaluated_score)/temp_dict[key]
        value=((previous_value*(temp_dict[key]-1))+float(evaluated_score))/temp_dict[key]
        print(value)
        state_action_dict.at[key,list2[i]]=value
        i=i+1
    

def question_or_not(words):
    if "?" not in words:
        q="q'"
    else:
        q="q"
    #print q
    return q

def count_no_words(words):
    no=len(words.split())
    if no>=10:
        #print "long"
        l="l"
    else:
        #print "short"
        l="l'"
    return l

def sentiment(words):
    blob = TextBlob(words)
    senti=blob.sentiment
    if senti.polarity<0.2 and senti.polarity>-0.2:
        s="s'"
    else:
        if senti.polarity>0:
            s="s+"
        else:
            s="s-"
    return s
    


def beginning_end(line,list):
    length=len(list)
    if list.index(line)<(0.6*length):
        #print "beginning"
        var="b"
    else:
        #print "ending"
        var="e"
    return var

def rows_utterance(list1):
    
    for values in list1:
        if question_or_not(values)=="q":
            if sentiment(values)=="s+":
                if beginning_end(values,list1)=="b":
                    result="qs+b"
                else:
                    result="qs-e"
            elif sentiment(values)=="s-":
                if beginning_end(values,list1)=="b":
                    result="qs-b"
                else:
                    result="qs-e"
            else:
                if beginning_end(values,list1)=="b":
                    result="qs'b"
                else:
                    result="qs'e"
        else:
            if count_no_words(values)=="l":
            
                if sentiment(values)=="s+":
                
                    if beginning_end(values,list1)=="b":
                        result="q'ls+b"
                    else:
                        result="q'ls+e"
                elif sentiment(values)=="s-":
                    if beginning_end(values,list1)=="b":
                        result="q'ls-b"
                    else:
                        result="q'ls-e"
                else:
                    if beginning_end(values,list1)=="b":
                        result="q'ls'b"
                    else:
                        result="q'ls'e"
            else:
                if sentiment(values)=="s+":
                
                    if beginning_end(values,list1)=="b":
                        result="q'l's+b"
                    else:
                        result="q'l's+e"
                elif sentiment(values)=="s-":
                    if beginning_end(values,list1)=="b":
                        result="q'l's-b"
                    else:
                        result="q'l's-e"
                else:
                    if beginning_end(values,list1)=="b":
                        result="q'l's'b"
                    else:
                        result="q'l's'e"
        user_utterances.append(result)
    return user_utterances

def main():
    # load strategies from pickle
    strats = pickle.load(open("strategies5.pkl", "rb"))

    try:
        # create file
        name = input("Before we start: what's your name? (Molly, Wei or Nehal)?\n")
        date = str(datetime.datetime.today().date())
        convo_num = get_convo_num(name)
        #filename = "./data/{0}_{1}_{2}.txt".format(name.lower(), date, convo_num) 
        #f = open(filename, "w")
    
        # record converstaion
        strategy = random.choice(list(strats.keys()))
        print("--------------------here it gones--------------------------------")
        
        log("Bot", random.choice(strats[strategy]), strategy)
        bot_strategies.append(strategy)
        for turn in range(N_TURN-1):
            bot_strategies.append(strategy)
            answer=str(input(name + ": "))
            log(name, answer, "None")
            list1.append(answer)
            strategy = random.choice(list(strats.keys()))
            log("Bot", random.choice(strats[strategy]), strategy)
        answer=str(input(name + ": "))
        log(name, answer, "None")
        list1.append(answer)
        
        # evaluation and wrap up
        score = input("\nHow is the conveseration? (number between 0 to 5) ")
        while ord(score) < ord("0") or ord(score) > ord("5"):
            score = input("Type a integer in [0,1], otherwise I will keep asking")
        #f.write("evaluation={0}".format(score))
        #f.close()
        print("Done!")
        print(list1)
        rows_utterance(list1)
        print(user_utterances)
        print(bot_strategies)
        
        record_check(user_utterances,bot_strategies,score)
        
    
    except KeyboardInterrupt:
        os.remove(filename)
        print("key board interupted, nothing is recorded")
    except:
        raise    


def get_convo_num(name):
    try:
        files = glob.glob("./data/{0}_*".format(name))
        existing_nums = list(map(int, [i[:-4].split("_")[-1] for i in files]))
        return max(existing_nums) + 1
    except ValueError:
        return 1
    
def log(role, utter, strategy):
    if role == "Bot":
        print("Bot says: ", utter)
    time = datetime.datetime.now().strftime("%H:%M:%S")
    #file.write("{0}_{1}_{2}:{3}\n".format(role, strategy, time, utter))

if __name__ == "__main__":
    main()

Before we start: what's your name? (Molly, Wei or Nehal)?
Nehal
--------------------here it gones--------------------------------
Bot says:  Can you be brave if you are afraid? That's the only time you can be brave
Nehal: Very true
Bot says:  That's neat!
Nehal: Thanks
Bot says:  Nope
Nehal: What did you mena?

How is the conveseration? (number between 0 to 5) 1
Done!
['Very true', 'Thanks', 'What did you mena?']
["q'l's+b", "q'l's+b", "qs'e"]
['opinion', 'opinion', 'affirmative']
1.0
0.5
1.0
0.666666666667
1.0
1.0


In [10]:
print(state_action_dict)
print(temp_dict)

         affirmative   opinion  negative  question
qs+b             0.0  0.000000       0.0       0.0
qs+e             0.0  0.000000       0.0       0.0
qs-b             0.0  0.000000       0.0       0.0
qs-e             0.0  0.000000       0.0       0.0
qs'b             1.0  0.000000       0.0       0.0
qs'e             1.0  0.000000       0.0       0.0
q'ls+b           0.0  0.000000       0.0       0.0
q'ls+e           0.0  0.000000       0.0       0.0
qls-b            0.0  0.000000       0.0       0.0
q'ls-e           0.0  0.000000       0.0       0.0
q'ls'b           0.0  0.000000       0.0       0.0
q'ls'e           0.0  0.000000       0.0       0.0
q'l's+b          1.0  0.666667       0.0       0.0
q'l's+e          1.0  0.000000       0.0       0.0
q'l's-b          0.0  0.000000       0.0       0.0
q'l's-e          0.0  0.000000       0.0       0.0
q'l's'b          0.5  0.000000       0.0       1.0
q'l's'e          0.0  0.500000       1.0       0.0
{'qs+b': 0, 'qs+e': 0, 'qs-b': 

In [9]:
list1=[]
user_utterances=[]
bot_strategies=[]
#storing the trained dataset values
def store(list1,list2,evaluated_score):
    i=0
    for key in list1:
        #print state_action_dict.at[key,list2[i]]
        previous_value=state_action_dict.at[key,list2[i]]
        temp_dict[key]=temp_dict[key]+1
        print(float(evaluated_score))
        #print temp_dict[key],((previous_value*(temp_dict[key]-1))+evaluated_score)/temp_dict[key]
        value=((previous_value*(temp_dict[key]-1))+float(evaluated_score))/temp_dict[key]
        print(value)
        state_action_dict.at[key,list2[i]]=value
        i=i+1
        
#learning after 10 conversations  
def check_and_pass(state_action_dict,sentence,list1):
    state=rows_utterance(sentence,list1)
    print("here")
    print(state)
    for value in state_action_dict:
        v=state_action_dict.at[state,value]
        if v==max(state_action_dict.loc[state]):
            print(value)
            break
    return value

def question_or_not(words):
    if "?" not in words:
        q="q'"
    else:
        q="q"
    #print q
    return q

def count_no_words(words):
    no=len(words.split())
    if no>=10:
        #print "long"
        l="l"
    else:
        #print "short"
        l="l'"
    return l

def sentiment(words):
    blob = TextBlob(words)
    senti=blob.sentiment
    if senti.polarity<0.2 and senti.polarity>-0.2:
        s="s'"
    else:
        if senti.polarity>0:
            s="s+"
        else:
            s="s-"
    return s
    


def beginning_end(line,list):
    length=len(list)
    if list.index(line)<(0.6*length):
        #print "beginning"
        var="b"
    else:
        #print "ending"
        var="e"
    return var

def form_user_utterancelist(list1):
    
    for values in list1:
        if question_or_not(values)=="q":
            if sentiment(values)=="s+":
                if beginning_end(values,list1)=="b":
                    result="qs+b"
                else:
                    result="qs-e"
            elif sentiment(values)=="s-":
                if beginning_end(values,list1)=="b":
                    result="qs-b"
                else:
                    result="qs-e"
            else:
                if beginning_end(values,list1)=="b":
                    result="qs'b"
                else:
                    result="qs'e"
        else:
            if count_no_words(values)=="l":
            
                if sentiment(values)=="s+":
                
                    if beginning_end(values,list1)=="b":
                        result="q'ls+b"
                    else:
                        result="q'ls+e"
                elif sentiment(values)=="s-":
                    if beginning_end(values,list1)=="b":
                        result="q'ls-b"
                    else:
                        result="q'ls-e"
                else:
                    if beginning_end(values,list1)=="b":
                        result="q'ls'b"
                    else:
                        result="q'ls'e"
            else:
                if sentiment(values)=="s+":
                
                    if beginning_end(values,list1)=="b":
                        result="q'l's+b"
                    else:
                        result="q'l's+e"
                elif sentiment(values)=="s-":
                    if beginning_end(values,list1)=="b":
                        result="q'l's-b"
                    else:
                        result="q'l's-e"
                else:
                    if beginning_end(values,list1)=="b":
                        result="q'l's'b"
                    else:
                        result="q'l's'e"
        user_utterances.append(result)
    return user_utterances

def rows_utterance(sentence,list1):
    print(list1)
    print("in here2")
    if question_or_not(sentence)=="q":
        if sentiment(sentence)=="s+":
            if beginning_end(sentence,list1)=="b":
                result="qs+b"
            else:
                result="qs-e"
        elif sentiment(sentence)=="s-":
            if beginning_end(sentence,list1)=="b":
                result="qs-b"
            else:
                result="qs-e"
        else:
            if beginning_end(sentence,list1)=="b":
                result="qs'b"
            else:
                result="qs'e"
    else:
        if count_no_words(sentence)=="l":
            
            if sentiment(sentence)=="s+":
                
                if beginning_end(sentence,list1)=="b":
                    result="q'ls+b"
                else:
                    result="q'ls+e"
            elif sentiment(sentence)=="s-":
                if beginning_end(sentence,list1)=="b":
                    result="q'ls-b"
                else:
                    result="q'ls-e"
            else:
                if beginning_end(sentence,list1)=="b":
                    result="q'ls'b"
                else:
                    result="q'ls'e"
        else:
            if sentiment(sentence)=="s+":
                
                if beginning_end(sentence,list1)=="b":
                    result="q'l's+b"
                else:
                    result="ql's+e"
            elif sentiment(sentence)=="s-":
                if beginning_end(sentence,list1)=="b":
                    result="q'l's-b"
                else:
                    result="q'l's-e"
            else:
                if beginning_end(sentence,list1)=="b":
                    result="q'l's'b"
                else:
                    result="q'l's'e"
        user_utterances.append(result)
        #print("in here3")
        #print("list"+user_utterances)
    return result



def main():
    # load strategies from pickle
    strats = pickle.load(open("strategies5.pkl", "rb"))

    try:
        # create file
        name = input("Before we start: what's your name? (Molly, Wei or Nehal)?\n")
        date = str(datetime.datetime.today().date())
        convo_num = get_convo_num(name)
        #filename = "./data/{0}_{1}_{2}.txt".format(name.lower(), date, convo_num) 
        #f = open(filename, "w")
    
        # record converstaion
        strategy = random.choice(list(strats.keys()))
        print("--------------------here it gones--------------------------------")
        
        log("Bot", random.choice(strats[strategy]), strategy)
        bot_strategies.append(strategy)
        for turn in range(N_TURN-1):
            bot_strategies.append(strategy)
            answer=str(input(name + ": "))
            log(name, answer, "None")
            list1.append(answer)
            print(answer)
            print(list1)
            strategy=check_and_pass(state_action_dict,answer,list1)
            print(strategy)
            bot_answer = random.choice(strats[strategy])
            log("Bot", bot_answer, strategy)
            bot_strategies.append(strategy)
        answer=str(input(name + ": "))
        log(name, answer, "None")
        list1.append(answer)
        
        # evaluation and wrap up
        score = input("\nHow is the conveseration? (number between 0 or 1) ")
        while ord(score) < ord("0") or ord(score) > ord("5"):
            score = input("Type a integer in [0,1], otherwise I will keep asking")
        #f.write("evaluation={0}".format(score))
        #f.close()
        print("Done!")
        #print(list1)
        form_user_utterancelist(list1)
        #print(user_utterances)
        #print(bot_strategies)
        store(user_utterances,bot_strategies,score)
        
    
    except KeyboardInterrupt:
        os.remove(filename)
        print("key board interupted, nothing is recorded")
    except:
        raise    


def get_convo_num(name):
    try:
        files = glob.glob("./data/{0}_*".format(name))
        existing_nums = list(map(int, [i[:-4].split("_")[-1] for i in files]))
        return max(existing_nums) + 1
    except ValueError:
        return 1
    
def log(role, utter, strategy):
    if role == "Bot":
        print("Bot says: ", utter)
    time = datetime.datetime.now().strftime("%H:%M:%S")
    #file.write("{0}_{1}_{2}:{3}\n".format(role, strategy, time, utter))

if __name__ == "__main__":
    main()



Before we start: what's your name? (Molly, Wei or Nehal)?
Nehal
--------------------here it gones--------------------------------
Bot says:  Yep!
Nehal: What?
What?
['What?']
['What?']
in here2
here
qs'b
affirmative
affirmative
Bot says:  Cool
Nehal: I didnt get you
I didnt get you
['What?', 'I didnt get you']
['What?', 'I didnt get you']
in here2
here
q'l's'b
question
question
Bot says:  Do you want to ride a dragon?
Nehal: Yes,i love to

How is the conveseration? (number between 0 or 1) 1
Done!
1.0
0.333333333333
1.0
1.0
1.0
0.5
1.0
1.0
